## A word on Interfaces

Note that my three different forests all have a common set of methods and properties/attributes that I can make use of.  This allows me to set up experiments like the following.  (Note that the code will not run yet in this notebook, as we have not defined the `BurnableForest`, `SlowBurnForest`, and `InstantBurnForest` classes.  Start executing cells from from the second code cell.)

In [ ]:
tree_history = []
fire_history = []
forests = [
    BurnableForest(),
    SlowBurnForest(),
    InstantBurnForest(),
]
num_generations = 2500
for i in xrange(num_generations):
    for forest in forests:
        forest.advance_one_step()
    tree_history.append(tuple(f.tree_fraction for f in forests))
    fire_history.append(tuple(f.fire_fraction for f in forests))

Note that it's three different classes, but they all have the same interface for doing the calculation.  Python doesn't do any checking for class inheritance to see whether a method is callable.  Instead, if there's a method with the right name, then it gets called.  This is the essence of what we call "Duck Typing".

Let's make this concrete and go back to our example `Leaf` class.

In [ ]:
class Leaf(object):
    def __init__(self, color="green"):
        self.color = color
    def fall(self):
        print "Splat"

class MapleLeaf(Leaf):
    def fall(self):
        self.color = "brown"
        super(MapleLeaf, self).fall()

class Acorn(object):
    def fall(self):
        print "Plunk"

In [ ]:
objects = [Leaf(), MapleLeaf(), Acorn()]
for obj in objects:
    obj.fall()

We know that we could make another kind of Forest by subclassing `Forest`, so let's do that.

While we're doing that, I want to introduce a minor refactoring of the Forest to create a `losses` attribute that we can use to generically track mold or fires or whatever future agent we can think of.

In [ ]:
import numpy as np
from matplotlib.pyplot import legend, plot
from scipy.ndimage.measurements import label
%matplotlib inline

class Forest(object):
    def __init__(self, size=(150, 150), p_sapling=0.0025, name=None):
        self.size = size
        self.trees = np.zeros(self.size, dtype=bool)
        self.p_sapling = p_sapling
        if name is not None:
            self.name = name
        else:
            self.name = self.__class__.__name__

    @property
    def num_cells(self):
        return self.size[0] * self.size[1]

    @property
    def tree_fraction(self):
        return self.trees.sum() / float(self.num_cells)

    @property
    def losses(self):
        """The base Forest class has no mechanism for losses.
        """
        return np.zeros(self.size)

    @property
    def loss_fraction(self):
        return self.losses.sum() / float(self.num_cells)

    def advance_one_step(self):
        self.grow_trees()

    def grow_trees(self):
        growth_sites = self._rand_bool(self.p_sapling)
        self.trees[growth_sites] = True

    def _rand_bool(self, p):
        return np.random.uniform(size=self.trees.shape) < p

Now we'll define our `BurnableForest` subclass with the `losses` property defined, which will be inherited by its subclasses.

In [ ]:
class BurnableForest(Forest):
    """This class has fires and a probability of lightning strike but no
    methods to burn the forest.
    """
    def __init__(self, p_lightning=5.0e-6, *args, **kwargs):
        super(BurnableForest, self).__init__(*args, **kwargs)
        self.p_lightning = p_lightning
        self.fires = np.zeros(self.size, dtype=bool)

    @property
    def fire_fraction(self):
        return self.fires.sum() / float(self.num_cells)

    @property
    def losses(self):
        return self.fires

class SlowBurnForest(BurnableForest):
    """BurnableForest where fires spread at the same time scale as the new
    trees grow.
    """
    def advance_one_step(self):
        self.grow_trees()
        self.start_fires()
        self.burn_trees()

    def burn_trees(self):
        # assumes 2D forest
        working_size = (self.size[0] + 2, self.size[1] + 2)
        fires = np.zeros(working_size, dtype=bool)
        fires[1:-1, 1:-1] = self.fires
        north = fires[:-2, 1:-1]
        south = fires[2:, 1:-1]
        east = fires[1:-1, :-2]
        west = fires[1:-1, 2:]
        new_fires = (north | south | east | west) & self.trees
        self.trees[self.fires] = False
        self.fires = new_fires

    def start_fires(self):
        lightning_strikes = self._rand_bool(self.p_lightning) & self.trees
        self.fires[lightning_strikes] = True

class InstantBurnForest(BurnableForest):
    """BurnableForest where fires spread instantaneously relative to the
    growth of new saplings.
    """
    def advance_one_step(self):
        self.grow_trees()
        self._strike_and_burn()

    def _strike_and_burn(self):
        strikes = self._rand_bool(self.p_lightning) & self.trees
        groves, num_groves = label(self.trees)
        fires = set(groves[strikes])
        self.fires.fill(False)
        for fire in fires:
            self.fires[groves == fire] = True
        self.trees[self.fires] = False

In [ ]:
# Try rerunning this cell with different values of p_lightning
p_lightning = 5.0e-6
forest = Forest()
burnable_forest = BurnableForest(p_lightning=p_lightning)
slow_burn_forest = SlowBurnForest(p_lightning=p_lightning)
inst_burn_forest = InstantBurnForest(p_lightning=p_lightning)
forests = [forest, burnable_forest, slow_burn_forest, inst_burn_forest]
names = [f.__class__.__name__ for f in forests]
loss_history = []

for i in xrange(1500):
    for fst in forests:
        fst.advance_one_step()
    loss_history.append([f.losses.sum() for f in forests])

plot(loss_history)
legend(names)

Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com